In [2]:
import pickle
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix

# Load models and data
with open('../models/good_model.pkl', 'rb') as f:
    good_model = pickle.load(f)
with open('../models/bad_model.pkl', 'rb') as f:
    bad_model = pickle.load(f)
with open('../data/test_data.pkl', 'rb') as f:
    test_data = pickle.load(f)

X_test = test_data['X_test']
y_test = test_data['y_test']
features = test_data['features']

def get_feature_importances(pipeline_or_model):
    if hasattr(pipeline_or_model, 'named_steps'):
        model = pipeline_or_model.named_steps['classifier']
        return model.feature_importances_
    else:
        return pipeline_or_model.feature_importances_

def test_location_importance(model, features):
    importances = get_feature_importances(model)
    
    # Get indices for location-related features
    neighborhood_indices = [i for i, f in enumerate(features) if 'buurt' in f]
    district_indices = [i for i, f in enumerate(features) if 'wijk' in f]
    rotterdam_indices = [i for i, f in enumerate(features) if 'rotterdam' in f]
    
    location_importance = {
        'neighborhood': sum(importances[i] for i in neighborhood_indices),
        'district': sum(importances[i] for i in district_indices),
        'rotterdam': sum(importances[i] for i in rotterdam_indices)
    }
    
    total_location_importance = sum(location_importance.values())
    total_model_importance = sum(importances)
    
    return {
        'location_importance_ratio': total_location_importance / total_model_importance,
        'location_breakdown': location_importance,
        'interpretation': f"Location features account for {(total_location_importance/total_model_importance)*100:.1f}% of model's decision making"
    }

if __name__ == "__main__":
    print("Testing good model:")
    good_results = test_location_importance(good_model, features)
    print(good_results['interpretation'])
    print("\nLocation breakdown:")
    for area, importance in good_results['location_breakdown'].items():
        print(f"{area}: {importance:.3f}")
    
    print("\nTesting bad model:")
    bad_results = test_location_importance(bad_model, features)
    print(bad_results['interpretation'])
    print("\nLocation breakdown:")
    for area, importance in bad_results['location_breakdown'].items():
        print(f"{area}: {importance:.3f}")

Testing good model:
Location features account for 5.9% of model's decision making

Location breakdown:
neighborhood: 0.005
district: 0.053
rotterdam: 0.000

Testing bad model:
Location features account for 6.4% of model's decision making

Location breakdown:
neighborhood: 0.001
district: 0.064
rotterdam: 0.000
